<div align="right">
  <img src="./imgs/EII-ULPGC-logo.jpeg" width="600">
</div>

# **Aprendizaje por refuerzo 5**

### **Métodos "Policy-Based" y "Value-Based"**

Los métodos **value-based**, como **Q-Learning** o **Deep Q-Learning**, se centran en aprender el valor de cada estado o estado-acción (la función $Q$) para tomar decisiones. Aunque eficientes, tienen limitaciones en entornos con acciones continuas o políticas complejas, donde la estimación de los valores puede ser difícil o inexacta.

### **Transición hacia los Métodos "Policy-Based"**

En lugar de estimar valores, los métodos **policy-based** (basados en políticas) **aprenden directamente una política que mapea estados a acciones**. En otras palabras, el agente aprende **qué acción** tomar en cada estado sin estimar previamente los valores de esos estados o acciones. 

### **Diferencias Clave**

1. **Optimización Directa de la Política**: Los métodos policy-based no calculan valores $ Q(s, a) $, sino que optimizan directamente la probabilidad de tomar acciones que maximizan la recompensa futura. Esto los hace especialmente útiles en entornos con **acciones continuas** y **políticas complejas**.
   
2. **Políticas Estocásticas**: En lugar de seleccionar siempre la mejor acción determinísticamente (como en los métodos value-based), los métodos policy-based aprenden una **política estocástica**, donde las acciones se seleccionan según una distribución de probabilidades. Esto facilita la exploración de diferentes estrategias.

3. **Uso del Gradiente**: En lugar de seguir una estrategia $ \epsilon $-greedy (como en Q-Learning), los métodos policy-based usan **gradientes de política** para ajustar la probabilidad de cada acción. Un ejemplo es el algoritmo **REINFORCE**, donde se actualizan los parámetros de la política en la dirección que maximiza la recompensa esperada.

### **Ventajas de los Métodos "Policy-Based"**

- **Acciones continuas**: Pueden manejar entornos con **espacios de acción continuos**, mientras que los métodos value-based requieren discretizar el espacio, lo que no siempre es eficiente.
- **Convergencia más estable**: Los métodos policy-based tienden a ser más estables en su convergencia, especialmente en entornos grandes o con variabilidad.
- **Exploración Implícita**: Al tener políticas estocásticas, la exploración está implícita en la política aprendida, sin necesidad de ajustar manualmente parámetros como $ \epsilon $.

### **Policy Gradient**

En **gradientes de política**, se maximiza el rendimiento esperado del agente ajustando los parámetros de la política en la dirección de los gradientes que indican un aumento en la recompensa. Esto se traduce en ajustar las probabilidades de seleccionar acciones que resultaron en buenos resultados. Un ejemplo clásico es el algoritmo **REINFORCE**:

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\pi_\theta} \left[ \nabla_\theta \log \pi_\theta(a | s) R \right]
$$

Donde $ R $ es la recompensa acumulada y $ \pi_\theta(a | s) $ es la probabilidad de tomar la acción $ a $ en el estado $ s $, dado los parámetros $ \theta $.


https://huggingface.co/learn/deep-rl-course/unit4/policy-gradient

https://huggingface.co/learn/deep-rl-course/unit4/pg-theorem

https://huggingface.co/learn/deep-rl-course/unit6/variance-problem

## **Profundizando en los métodos de gradiente de política**

### **Visión general**

Los métodos de gradiente de política buscan encontrar parámetros que **maximicen el retorno esperado**. 

La idea es que tenemos una *política estocástica parametrizada*. En nuestro caso, una red neuronal produce una distribución de probabilidad sobre las acciones. La probabilidad de tomar cada acción también se llama *preferencia de acción*.

Tomando el ejemplo de CartPole-v1:

- Como entrada, tenemos un estado.
- Como salida, tenemos una distribución de probabilidad sobre las acciones en ese estado.

Nuestro objetivo con el gradiente de política es **controlar la distribución de probabilidad de las acciones** ajustando la política para que **las buenas acciones (que maximizan el retorno) se muestreen con más frecuencia en el futuro**.

Cada vez que el agente interactúa con el entorno, ajustamos los parámetros para que las buenas acciones sean más probables en el futuro.

Pero **¿cómo vamos a optimizar los pesos usando el retorno esperado?**

La idea es **dejar que el agente interactúe durante un episodio**. Y si ganamos el episodio, consideramos que cada acción tomada fue buena y debe muestrearse más en el futuro, ya que llevaron a ganar.

Así que, para cada par estado-acción querremos incrementar la $P(a|s)$ : la probabilidad de tomar la acción a en el estado s. O decrementarla si la acción no fue buena.

El algoritmo policy-gradient simplificado sería:

```raw
Repetir
  Recopilar un episodio con la poliítica actual
  Calcular el retorno esperado (suma de recompensas)
  Actualizar los pesos de la política:
    Incrementar la probabilidad de la acción de cada par estado-acción si el retorno fue alto
    Decrementar la probabilidad de la acción de cada par estado-acción si el retorno fue bajo
```

Entremos ahora en los detalles del método.

Tenemos nuestra política estocástica $\pi$ parametrizada por $\theta$.

$$
\pi_\theta(s) = P(A|s;\theta)
$$

Dado un estado, la política nos da una distribución de probabilidad sobre las acciones. Pero, ¿cómo sabemos si nuestra política es buena? Necesitamos una forma de medirla. Para ello, definimos una función objetivo o score que llamamos $J(\theta)$.

### **La función objetivo**

La *función objetivo* nos da el **rendimiento del agente** dada una trayectoria, y produce el *retorno acumulado esperado*.

$$
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} [G(\tau)]
$$

Donde $\tau$ es una trayectoria (o episodio) y $G(\tau)$ es el retorno acumulado esperado.

$$
G(\tau) = \sum_{t=0}^{T} \gamma^t r_t
$$

Veamos más detalles sobre esta fórmula:

El **retorno esperado** (también llamado recompensa acumulada esperada) es el promedio ponderado (donde los pesos están dados por $ P(\tau; \theta) $) de todos los posibles valores que el retorno $ G(\tau) $ puede tomar.

Este concepto es clave para evaluar cómo de buena es una política en función de las recompensas futuras que puede obtener el agente.

---------------
---------------



Nuestro objetivo es maximizar el retorno acumulado esperado encontrando el θ que produzca las mejores distribuciones de probabilidad de acción:

$$\theta^* = \arg\max_\theta J(\theta)$$

## Ascenso de gradiente y el teorema del gradiente de política

El gradiente de política es un problema de optimización: queremos encontrar los valores de θ que maximicen nuestra función objetivo J(θ), así que necesitamos usar **ascenso de gradiente**[1].

## El algoritmo REINFORCE (Monte Carlo REINFORCE)

El algoritmo REINFORCE, también llamado gradiente de política de Monte Carlo, es un algoritmo de gradiente de política que **usa un retorno estimado de un episodio completo para actualizar el parámetro de política** θ:

En un bucle:
1. Usar la política para recopilar un episodio
2. Usar el episodio para estimar el gradiente
3. Actualizar los pesos de la política:

   $$\theta \leftarrow \theta + \alpha \nabla_\theta J(\theta)$$

Podemos interpretar esta actualización de la siguiente manera:

- ∇θ log πθ(at|st) es la dirección del **aumento más pronunciado de la (log) probabilidad** de seleccionar la acción a desde el estado s.

- G(τ) es la función de puntuación:
  - Si el retorno es alto, **aumentará las probabilidades** de las combinaciones (estado, acción).
  - Si el retorno es bajo, **disminuirá las probabilidades** de las combinaciones (estado, acción)[1].

También podemos **recopilar múltiples episodios (trayectorias)** para estimar el gradiente.

Citations:
[1] https://huggingface.co/learn/deep-rl-course/unit4/policy-gradient